In [11]:
import pickle
import random
import pandas as pd
from tqdm import tqdm
dataset_name = 'askubuntu_stackexchange'

s_2_rot, o_2_rst, so_2_rt = {}, {}, {}
all_samples = []
edge_list = pd.read_csv(dataset_name + '/' + 'edge_list.csv')
for inx, row in edge_list.iterrows():
    s, r, o, t, l = row['u'], row['r'], row['i'], row['ts'], row['label']
    s, r, o, t, l = int(s), int(r), int(o), int(t), int(l)
    all_samples.append((s, r, o, l, t))
    if s not in s_2_rot.keys():
        s_2_rot[s] = []
    s_2_rot[s].append([r,o,l,t])

    if o not in o_2_rst.keys():
        o_2_rst[o] = []
    o_2_rst[o].append([r,s,l,t])

    if (s,o) not in so_2_rt.keys():
        so_2_rt[(s,o)] = []
    so_2_rt[(s,o)].append([r,l,t])

test_samples = all_samples


In [12]:
pair_set = set()
for key in so_2_rt.keys():
    if len(so_2_rt[key]) >= 2:
        pair_set.add(key)
print(len(pair_set))

270898


In [13]:
test_set_num = 1000

test_set = {} # (s,r,o,t,l) : sorted_shis[], sorted_ohis[], sorted_pairhis[]

print(len(test_samples))
random.shuffle(test_samples)

for sample in test_samples:
    s, r, o, l, t = sample
    if (s,o) in pair_set:
        pair_his = []
        for pair_his_sample in so_2_rt[(s,o)]:
            his_r, his_l, his_t = pair_his_sample
            if his_t < t:
                pair_his.append([s, his_r, o, his_l, his_t])
        if len(pair_his) == 0:
            continue
        else:
            pair_his = sorted(pair_his, key=lambda k: k[-1], reverse=True)
    else:
        continue
        
    s_his = []
    for s_his_sample in s_2_rot[s]:
        his_r, his_o, his_l, his_t = s_his_sample
        if his_t < t:
            s_his.append([s, his_r, his_o, his_l, his_t])
    if len(s_his) == 0:
        continue
    else:
        s_his = sorted(s_his, key=lambda k: k[-1], reverse=True)

    o_his = []
    for o_his_sample in o_2_rst[o]:
        his_r, his_s, his_l, his_t = o_his_sample
        if his_t < t:
            o_his.append([his_s, his_r, o, his_l, his_t])
    if len(o_his) == 0:
        continue
    else:
        o_his = sorted(o_his, key=lambda k: k[-1], reverse=True)
    
    test_set[(s, r, o, l, t)] = [s_his, o_his, pair_his]
    if len(test_set) == test_set_num:
        break

ind = 0
print(len(test_set))
print(list(test_set.keys())[ind])
print(test_set[list(test_set.keys())[ind]])
pickle.dump(test_set, open(dataset_name + '/' + 'LLM_test.pkl', 'wb'))

1497006
1000
(9628, 1388282, 289868, 0, 4485)
[[[9628, 1387762, 289868, 0, 4482], [9628, 1387765, 289868, 0, 4482], [9628, 1386866, 441302, 0, 4479], [9628, 1385658, 670732, 0, 4475], [9628, 1384714, 224330, 0, 4472], [9628, 1384716, 392189, 0, 4472], [9628, 1384716, 559763, 0, 4472], [9628, 1384576, 197942, 0, 4471], [9628, 1343463, 363475, 0, 4292], [9628, 1326556, 255028, 1, 4223], [9628, 1326558, 549285, 0, 4223], [9628, 1297948, 486322, 0, 4111], [9628, 1126340, 259309, 0, 3568], [9628, 1126370, 467975, 0, 3568], [9628, 1126371, 467975, 0, 3568], [9628, 1126396, 467975, 0, 3568], [9628, 1125851, 259309, 0, 3567], [9628, 1104351, 570278, 0, 3503], [9628, 1089296, 672456, 0, 3448], [9628, 1075842, 287093, 1, 3404], [9628, 1071909, 319306, 0, 3391], [9628, 1071910, 319306, 0, 3391], [9628, 1071911, 319306, 0, 3391], [9628, 1061531, 384215, 0, 3351], [9628, 958180, 216405, 0, 3006], [9628, 958182, 465761, 0, 3006], [9628, 958185, 343627, 0, 3006], [9628, 806807, 564778, 0, 2578], [962

In [14]:
train_set_num = 10000
train_set = {}

print(len(test_samples))
random.shuffle(test_samples)

for step in [0,1]:
    for sample in test_samples:
        if sample in test_set.keys() or sample in train_set.keys():
            continue
        s, r, o, l, t = sample
        if (s,o) in pair_set:
            pair_his = []
            for pair_his_sample in so_2_rt[(s,o)]:
                his_r, his_l, his_t = pair_his_sample
                if his_t < t:
                    pair_his.append([s, his_r, o, his_l, his_t])
            if len(pair_his) == 0:
                if step == 0:
                    continue
                else:
                    pair_his = []
            else:
                pair_his = sorted(pair_his, key=lambda k: k[-1], reverse=True)
        else:
            if step == 0:
                continue
            else:
                pair_his = []
            
        s_his = []
        for s_his_sample in s_2_rot[s]:
            his_r, his_o, his_l, his_t = s_his_sample
            if his_t < t:
                s_his.append([s, his_r, his_o, his_l, his_t])
        if len(s_his) == 0:
            continue
        else:
            s_his = sorted(s_his, key=lambda k: k[-1], reverse=True)

        o_his = []
        for o_his_sample in o_2_rst[o]:
            his_r, his_s, his_l, his_t = o_his_sample
            if his_t < t:
                o_his.append([his_s, his_r, o, his_l, his_t])
        if len(o_his) == 0:
            continue
        else:
            o_his = sorted(o_his, key=lambda k: k[-1], reverse=True)
        
        train_set[(s, r, o, l, t)] = [s_his, o_his, pair_his]
        if len(train_set) >= train_set_num:
            break
    if len(train_set) >= train_set_num:
            break

ind = 1
print(len(train_set))
print(list(train_set.keys())[ind])
print(train_set[list(train_set.keys())[ind]])
pickle.dump(train_set, open(dataset_name + '/' + 'LLM_train.pkl', 'wb'))

1497006
10000
(44227, 239969, 299478, 0, 1124)
[[[44227, 239777, 299478, 0, 1123], [44227, 239637, 299478, 0, 1122], [44227, 239638, 265517, 0, 1122], [44227, 239643, 265517, 0, 1122], [44227, 239648, 299478, 0, 1122], [44227, 166085, 378635, 0, 877], [44227, 166008, 378635, 0, 876], [44227, 130743, 316075, 0, 752], [44227, 130745, 316075, 0, 752], [44227, 130747, 316075, 0, 752], [44227, 130749, 316075, 0, 752], [44227, 130750, 316075, 0, 752], [44227, 130753, 316075, 0, 752], [44227, 130754, 316075, 0, 752], [44227, 130757, 316075, 0, 752], [44227, 130760, 316075, 0, 752], [44227, 130762, 316075, 0, 752]], [[293, 239693, 299478, 0, 1123], [44227, 239777, 299478, 0, 1123], [293, 239778, 299478, 0, 1123], [44227, 239637, 299478, 0, 1122], [44227, 239648, 299478, 0, 1122]], [[44227, 239777, 299478, 0, 1123], [44227, 239637, 299478, 0, 1122], [44227, 239648, 299478, 0, 1122]]]
